# Auto Regressive Model

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
from pymongo import MongoClient
from sklearn.metrics import mean_absolute_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.ar_model import AutoReg

# 1. Prepare Data
## 1.1. Import

<b>Task 3.3.1: Complete to the create a client to connect to the MongoDB server, assigns the "air-quality" database to db, and assigned the "nairobi" connection to nairobi.</b>

In [ ]:
client = MongoClient(host="localhost",port=27017)
db = client["air_quality"]
nairobi = db["nairobi"]

<b>Task 3.3.2: Change the wrangle function below so that it returns a Series of the resampled data instead of a DataFrame.</b>

In [ ]:
def wrangle(collection):
    results = collection.find(
        {"metadata.site": 29, "metadata.measurement": "P2"},
        projection={"P2": 1, "timestamp": 1, "_id": 0},
    )

    # Read data into DataFrame
    df = pd.DataFrame(list(results)).set_index("timestamp")

    # Localize timezone
    df.index = df.index.tz_localize("UTC").tz_convert("Africa/Nairobi")

    # Remove outliers
    df = df[df["P2"] < 500]

    # Resample to 1hr window
    y = df["P2"].resample("1H").mean().fillna(method='ffill').to_frame()

    return y

<b>Task 3.3.3: Use your wrangle function to read the data from the nairobi collection into the Series y.</b>

# 1.2. Explore

#  ACF Plot
When we've worked with autocorrelations in the past, we've treated them like static relationships, but that's not always how they work. Sometimes, we want to actually see how those autocorrelations change over time, which means we need to think of them as functions. When we create a visual representation of an autocorrelation function (ACF), we're making an ACF plot.

<b>Task 3.3.4: Create an ACF plot for the data in y. Be sure to label the x-axis as "Lag [hours]" and the y-axis as "Correlation Coefficient".</b>

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
plot_acf(y,ax=ax)

plt.xlabel("Lag [hours]")
plt.ylabel("Correlation Coefficient");

<b>Task 3.3.5: Create an PACF plot for the data in y. Be sure to label the x-axis as "Lag [hours]" and the y-axis as "Correlation Coefficient".</b>

# PACF Plot
Autocorrelations take into account two types of observations. Direct observations are the ones that happen exactly at our chosen time-step interval; we might have readings at one-hour intervals starting at 1:00. Indirect observations are the ones that happen between our chosen time-step intervals, at time-steps like 1:38, 2:10, 3:04, etc. Those indirect observations might be helpful, but we can't be sure about that, so it's a good idea to strip them out and see what our graph looks like when it's only showing us direct observations.

An autocorrelation that only includes the direct observations is called a partial autocorrelation, and when we view that partial autocorrelation as a function, we call it a PACF.

PACF plots represent those things visually. We want to compare our ACF and PACF plots to see which model best describes our time series. If the ACF data drops off slowly, then that's going to be a better description; if the PACF falls off slowly, then that's going to be a better description.

In [ ]:
y.shift(1).corr(y.shift(2))

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
plot_pacf(y,ax=ax)
plt.xlabel("Lag [hours]")
plt.ylabel("Correlation Coefficient");

# 1.3. Split

<b>Task 3.3.6: Split y into training and test sets. The first 95% of the data should be in your training set. The remaining 5% should be in the test set.</b>

In [ ]:
cutoff_test = int(len(y) * 0.95)

y_train = y.iloc[:cutoff_test]
y_test = y.iloc[cutoff_test:]

# Build Model
## Baseline
<b>Task 3.3.7: Calculate the baseline mean absolute error for your model.</b>

In [ ]:
y_train_mean = y_train.mean()
y_pred_baseline = [y_train_mean] * len(y_train)
mae_baseline = mean_absolute_error(y_train, y_pred_baseline)

print("Mean P2 Reading:", round(y_train_mean, 2))
print("Baseline MAE:", round(mae_baseline, 2))

# Iterate

<b>Task 3.3.8: Instantiate an AutoReg model and fit it to the training data y_train. Be sure to set the lags argument to 26</b>

In [ ]:
model = AutoReg(y_train, lags=26).fit()

<b>Task 3.3.9: Generate a list of training predictions for your model and use them to calculate your training mean absolute error.</b>

In [ ]:
y_pred = model.predict().dropna()
training_mae = mean_absolute_error(y_train.iloc[26:],y_pred)
print("Training MAE:", training_mae)

<b>Task 3.3.10: Use y_train and y_pred to calculate the residuals for your model.</b>

In [ ]:
y_train_resid = y_train-y_pred
y_train_resid.tail()

<b>Task 3.3.11: Create a plot of y_train_resid.</b>

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
y_train_resid.plot(ylabel="Residual Value",ax=ax);

<b>Task 3.3.12: Create a histogram of y_train_resid.</b>

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
y_train_resid.plot(kind="hist",ax=ax);
plt.ylabel("Residual Value")
plt.xlabel("Frequency")
plt.title("AR(26) Distribution of Residual");

<b>Task 3.3.13: Create an ACF plot of y_train_resid.</b>

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))   #have no more power of prediction
plot_acf(y_train_resid, ax=ax);

# Evaluate

<b>Task 3.3.14: Calculate the test mean absolute error for your model.</b>

In [ ]:
y_pred_test = model.predict(y_test.index.min(), y_test.index.max())
test_mae = mean_absolute_error(y_test,y_pred_test)
print("Test MAE:", test_mae)

<b>Task 3.3.15: Create a DataFrame test_predictions that has two columns: "y_test" and "y_pred". The first should contain the true values for your test set, and the second should contain your model's predictions. Be sure the index of test_predictions matches the index of y_test.</b>

In [ ]:
df_pred_test = pd.DataFrame(
    {"y_test": y_test, "y_pred": y_pred_test}, index=y_test.index
)

</b>Task 3.3.16: Create a time series plot for the values in test_predictions using plotly express. Be sure that the y-axis is properly labeled as "P2".<b>

In [ ]:
fig = px.line(df_pred_test, labels={"value": "P2"})
fig.show()

<b>Task 3.3.17: Perform walk-forward validation for your model for the entire test set y_test. Store your model's predictions in the Series y_pred_wfv.</b>

# Walk-Forward Validation
Our predictions lose power over time because the model gets farther and farther away from its beginning. But what if we could move that beginning forward with the model? That's what walk-forward validation is. In a walk-forward validation, we re-train the model at for each new observation in the dataset, dropping the data that's the farthest in the past. Let's say that our prediction for what's going to happen at 12:00 is based on what happened at 11:00, 10:00, and 9:00. When we move forward an hour to predict what's going to happen at 1:00, we only use data from 10:00, 11:00, and 12:00, dropping the data from 9:00 because it's now too far in the past

In [ ]:
%%capture

y_pred_wfv = pd.Series()
history = y_train.copy()
for i in range(len(y_test)):
    model = AutoReg(history, lags=26).fit()
    next_pred = model.forecast()
    y_pred_wfv = y_pred_wfv.append(next_pred)
    histroy = history.append(y_test[next_pred.index])

<b>Task 3.3.18: Calculate the test mean absolute error for your model.</b>

In [ ]:
test_mae = mean_absolute_error(y_test ,y_pred_wfv)
print("Test MAE (walk forward validation):", round(test_mae, 2))

<b>Task 3.3.19: Print out the parameters for your trained model.</b>

In [ ]:
print(model.params)

<b>Task 3.3.20: Put the values for y_test and y_pred_wfv into the DataFrame df_pred_test (don't forget the index). Then plot df_pred_test using plotly express.</b>

In [ ]:
df_pred_test = pd.DataFrame(
    {"y_test":y_test, "y_pred_wfv": y_pred_wfv}
)
fig = px.line(df_pred_test, labels={"value":"PM2.5"})
fig.show()